In [3]:
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import numpy as np
import json
import sqlite3
import datetime
from dateutil.relativedelta import relativedelta


HALL_NAME = "第一プラザ狭山店"
HALL_NAME = "パラッツォ川越店"
HALL_NAME = "パールショップともえ川越店"
HALL_NAME = "第一プラザ坂戸1000"
# HALL_NAME = "EXA FIRST"

DB_PATH = r"C:\python\dataOnline\anaslo_02\db\anaslo_02.db"

# spreadSheet_ids = {
#     "EXA FIRST": "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8",
    # "パラッツォ川越店": "179nJF0NvLng7xPKsd_NX2pJBXsDNsO8SJhOvUAvFk2I",
    # "第一プラザ狭山店": "1IVb2Woq3n_PDZP87LdW9NpFP3Z6LeyQtONCkx_fWIq4",
    # "パールショップともえ川越店": "1i70joJ27Hs7inS-D89z9YMSJO1aRvaBeeWn0n9xpktY",
    # }

# 検索キーワードよりホール名取得
# SPREADSHEET_ID = spreadSheet_ids[HALL_NAME]

# スプレッドシート認証設定
# scope = [
#     "https://spreadsheets.google.com/feeds",
#     "https://www.googleapis.com/auth/drive",
# ]
# jsonf = r"C:\python\dataOnline\anaslo_02\json\spreeadsheet-347321-ff675ab5ccbd.json"
# creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
# client = gspread.authorize(creds)
# spreadsheet = client.open_by_key(SPREADSHEET_ID)

# # Table name 取得
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables = cursor.fetchall()
# print(tables)

cursor.execute(
    "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + HALL_NAME + "%",)
)
results = cursor.fetchall()

# 結果表示
if results:
    for hall_id, hall_name in results:
        # print(f" - hall_name: {hall_name}, hall_id: {hall_id}")
        print(f"🔍 '{HALL_NAME}' を含むホール名が見つかりました。")
else:
    print(f"❌ '{HALL_NAME}' を含むホール名は見つかりませんでした。")

query = """
-- 出玉データにホール名と機種名を結合して取得
SELECT
    r.*, 
    h.name AS hall_name,     -- ホール名を追加
    m.name AS model_name     -- 機種名を追加
FROM results r
JOIN halls h ON r.hall_id = h.hall_id  -- ホールと結合
JOIN models m ON r.model_id = m.model_id  -- 機種と結合
WHERE h.name = ?  -- 指定ホールのみ
AND m.name LIKE '%ジャグラー%'  -- ジャグラー系機種に限定
ORDER BY r.date DESC, r.unit_no ASC;
"""

df = pd.read_sql_query(query, conn, params=(hall_name,))
conn.close()

# ブドウシミュレーター
def grape_calculator_myfive(game, bb, rb, medals, cherry=True):
    bb_medals = 239.25
    rb_medals = 95.25
    replay_rate = 0.411
    if cherry:
        cherry_rate_high = 0.04228
    else:
        cherry_rate_high = 0.05847
    denominator_inner = (-medals - (game*3 - (bb*bb_medals + rb*rb_medals + game*replay_rate + game*cherry_rate_high))) / 8
    grape_rate = (game / denominator_inner) - ((game / denominator_inner) * 2)
    
    return grape_rate

def assign_area(unit_no, json_file_path):
    with open(json_file_path, "r", encoding="utf-8") as f:
        area_map = json.load(f)
    for rule in area_map:
        if rule["start"] <= unit_no <= rule["end"]:
            return rule["area"]
    return "その他"

df["date"] = pd.to_datetime(df["date"])
df.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
df = df[["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]]
df["BB_rate"] = (df["game"] / df["BB"]).round(1)
df["RB_rate"] = (df["game"] / df["RB"]).round(1)
df["Grape_rate"] = grape_calculator_myfive(df["game"], df["BB"], df["RB"], df["medals"], cherry=True).round(2)
df["Total_rate"] = (df["game"] / (df["BB"] + df["RB"])).round(1)
df["month"] = df["date"].dt.strftime("%Y-%m")
df["day"] = df["date"].dt.day
df["weekday"] = df["date"].dt.weekday
df["unit_last"] = df["unit_no"].astype(str).str[-1]

JSON_FILE_PATH = r"C:\python\dataOnline\anaslo_02\json\exa_area_map.json"
df["area"] = df["unit_no"].apply(lambda x: assign_area(x, JSON_FILE_PATH))

df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0)

print(f'データサイズ: {df.shape[0]} x {df.shape[1]}')
print(f'以下の日付のデータが含まれています')
print(df.date.unique()[0])
model_list = list(df["model_name"].unique())
print(f'以下のモデルが含まれています')
for i, model in enumerate(model_list):
    print(f'{i}: {model}', end=", ")
    
df.head()

🔍 '第一プラザ坂戸1000' を含むホール名が見つかりました。
データサイズ: 11013 x 17
以下の日付のデータが含まれています
2025-04-26 00:00:00
以下のモデルが含まれています
0: アイムジャグラーEX-TP, 1: マイジャグラーV, 2: ゴーゴージャグラー3, 3: ジャグラーガールズ, 4: ファンキージャグラー2, 5: ハッピージャグラーVIII, 6: ウルトラミラクルジャグラー, 

,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,Grape_rate,Total_rate,month,day,weekday,unit_last,area
0,第一プラザ坂戸1000,2025-04-26,アイムジャグラーEX-TP,663,890,4,2,-64,222.5,445.0,6.75,148.3,2025-04,26,5,3,その他
1,第一プラザ坂戸1000,2025-04-26,アイムジャグラーEX-TP,664,2307,10,5,424,230.7,461.4,5.38,153.8,2025-04,26,5,4,その他
2,第一プラザ坂戸1000,2025-04-26,アイムジャグラーEX-TP,665,1256,6,2,331,209.3,628.0,5.28,157.0,2025-04,26,5,5,その他
3,第一プラザ坂戸1000,2025-04-26,アイムジャグラーEX-TP,666,352,0,1,-343,0.0,352.0,6.15,352.0,2025-04,26,5,6,その他
4,第一プラザ坂戸1000,2025-04-26,マイジャグラーV,789,4757,14,15,-1360,339.8,317.1,6.37,164.0,2025-04,26,5,9,その他


In [4]:
# 各種ピボットテーブル
def new_func(df, model_name, start_date, end_date):
    df_filtered = df.copy()
    df_filtered = df_filtered[df_filtered["model_name"] == model_name]
    df_filtered = df_filtered[(df_filtered["date"].dt.date >= start_date) & (df_filtered["date"].dt.date <= end_date)]
    pivot_targets = ["game", "RB_rate", "Total_rate"]
    pivot_results = {}
    for col in pivot_targets:
        table = df_filtered.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values=col,
        aggfunc="sum",
    )
        # 日付列を反転・スライス
        pivot_results[col] = table.iloc[:, ::-1]

    game = pivot_results["game"]
    rb_rate = pivot_results["RB_rate"]
    total_rate = pivot_results["Total_rate"]

    # labeled_tables = [("GAME", game), ("RB_RATE", rb_rate), ("TOTAL_RATE", total_rate)]

    # ラベルを MultiIndex に付ける
    # for label, df_table in labeled_tables:
    #     df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # display(game.head(2))
    # display(rb_rate.head(2))
    # display(total_rate.head(2))

    # 列を交互に整列して統合・NaN除去
    # interleaved_cols = [col for pair in zip(
    #     game.columns,
    #     rb_rate.columns,
    #     total_rate.columns,
    # ) for col in pair]
    # merged = pd.concat([game, rb_rate, total_rate], axis=1)[interleaved_cols]
    
    return game, rb_rate, total_rate

end_date = datetime.date.today()
start_date =  end_date - relativedelta(days=7)
game, rb_rate, total_rate = new_func(df, "マイジャグラーV", start_date, end_date)
game.head(2)

date                2025-04-26  2025-04-25  2025-04-24  2025-04-23  2025-04-21
model_name unit_no                                                            
マイジャグラーV   789            4757        3981        1724        4165        7110
           790            8908        5013        5471        7720        7329

In [31]:
model_list = [
    "マイジャグラーV",
    "ゴーゴージャグラー3",
    "アイムジャグラーEX-TP",
    "ファンキージャグラー2",
    "ミスタージャグラー",
    # "ウルトラミラクルジャグラー",
    # "ハッピージャグラーVIII",
    # "ジャグラーガールズ",
]

end_date = datetime.date.today()
start_date =  end_date - relativedelta(days=30)

df_filtered = df.copy()
merged_models = pd.DataFrame()
for model in list(model_list):
    print(model)
    empty_index = pd.MultiIndex.from_tuples([("", "")], names=game.index.names)
    empty_row = pd.DataFrame([[""] * game.shape[1]], index=empty_index, columns=game.columns)
    game, rb_rate, total_rate = new_func(df_filtered, model, start_date, end_date)
    merged_model = pd.concat([game, empty_row, rb_rate, empty_row, total_rate, empty_row], axis=0)
    # display(merged_model)
    merged_models = pd.concat([merged_models, merged_model, empty_row])

merged_models = merged_models[~merged_models.iloc[:, 0].isna()]
merged_models.to_csv(f"{HALL_NAME}_data.csv")
    
merged_models

マイジャグラーV
ゴーゴージャグラー3
アイムジャグラーEX-TP
ファンキージャグラー2
ミスタージャグラー


date               2025-04-26 2025-04-25 2025-04-24 2025-04-23 2025-04-22  \
model_name unit_no                                                          
マイジャグラーV   1021        8826.0     6716.0     3966.0     2325.0      917.0   
           1022        6063.0     2819.0     2040.0     1446.0     6283.0   
           1023        8608.0     5793.0     1164.0     1432.0     4225.0   
           1024        6370.0     6651.0     1039.0     2857.0     1629.0   
           1025        8163.0     6058.0     6269.0     1046.0     2006.0   
...                       ...        ...        ...        ...        ...   
ミスタージャグラー  1111         173.3      124.3      132.7      225.0      139.1   
           1112         184.4      146.2      146.6      261.0      161.5   
           1113         145.6      140.2      184.5      138.9      193.4   
                                                                            
                                                                            

date               2025-04-21 2025-04-20 2025-04-19 2025-04-18 2025-04-17  \
model_name unit_no                                                          
マイジャグラーV   1021        5682.0     2481.0     6266.0     6727.0     2756.0   
           1022        3960.0     5519.0     4189.0      638.0     4361.0   
           1023        1869.0     4971.0     6751.0     5084.0     3050.0   
           1024        7437.0     2858.0     4017.0     2041.0     4721.0   
           1025        3736.0     5616.0     2272.0     5285.0     3611.0   
...                       ...        ...        ...        ...        ...   
ミスタージャグラー  1111         170.7        NaN        NaN        NaN        NaN   
           1112         127.6        NaN        NaN        NaN        NaN   
           1113         121.6        NaN        NaN        NaN        NaN   
                                                                            
                                                                            

date                ... 2025-04-06 2025-04-05 2025-04-04 2025-04-03  \
model_name unit_no  ...                                               
マイジャグラーV   1021     ...     5007.0     6008.0     7051.0     4091.0   
           1022     ...     2808.0     8370.0     6547.0     1526.0   
           1023     ...     1549.0     6244.0     2721.0     2807.0   
           1024     ...     4499.0     7279.0     1175.0     5596.0   
           1025     ...     1243.0     6282.0     5857.0     6759.0   
...                 ...        ...        ...        ...        ...   
ミスタージャグラー  1111     ...        NaN        NaN        NaN        NaN   
           1112     ...        NaN        NaN        NaN        NaN   
           1113     ...        NaN        NaN        NaN        NaN   
                    ...                                               
                    ...                                               

date               2025-04-02 2025-04-01 2025-03-31 2025-03-30 2025-03-29  \
model_name unit_no                                                          
マイジャグラーV   1021        2288.0     3920.0     4091.0     3856.0     2173.0   
           1022        2897.0     5858.0     1526.0     5263.0     3833.0   
           1023        3713.0     5756.0     2807.0     1453.0     4913.0   
           1024        1492.0     2915.0     5596.0     7545.0     6980.0   
           1025        3606.0     4900.0     6759.0     7105.0     1361.0   
...                       ...        ...        ...        ...        ...   
ミスタージャグラー  1111           NaN        NaN        NaN        NaN        NaN   
           1112           NaN        NaN        NaN        NaN        NaN   
           1113           NaN        NaN        NaN        NaN        NaN   
                                                                            
                                                                            

date               2025-03-28  
model_name unit_no             
マイジャグラ